##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# サブクラス化によるレイヤとモデルの新規作成

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_layers_and_models"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/keras-team/keras-io/blob/master/tf/custom_layers_and_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/making_new_layers_and_models_via_subclassing.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/keras-io/tf/custom_layers_and_models.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## セットアップ

In [ ]:
import tensorflow as tf
from tensorflow import keras

## `Layer` クラス：状態（重み）といくつかの計算の組み合わせ

Kerasの中心的な抽象概念の1つは、`Layer`クラスです。 レイヤーは、状態（レイヤーの「重み」）と入力から出力への変換（「呼び出し」、レイヤーのフォワードパス）をカプセル化します。

以下の密に接続されたレイヤーで、状態は変数`w`と`b`をもちます。

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


Python関数のように、レイヤーをテンソル入力で呼び出して使用します。

In [ ]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

重み`w`および`b`をレイヤー属性として設定すると、レイヤーによって自動的に追跡されるので注意してください。

In [ ]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

また、`add_weight()`を使用すると、レイヤーにより迅速に重みを追加することができます。

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

## トレーニングできない重みをレイヤーに追加

トレーニング可能な重みだけでなく、トレーニング不可能な重みもレイヤーに追加できます。このような重みは、レイヤーをトレーニングする際、バックプロパゲーションで考慮されません。

以下のようにトレーニング不可能な重みを追加して使用します。

In [ ]:
class ComputeSum(keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total


x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

これは`layer.weights`の一部ですが、トレーニング不可能な重みとして分類されます。

In [ ]:
print("weights:", len(my_sum.weights))
print("non-trainable weights:", len(my_sum.non_trainable_weights))

# It's not included in the trainable weights:
print("trainable_weights:", my_sum.trainable_weights)

## ベストプラクティス：入力の形状がわかるまで重みづけを延期する

上記の`Linear` レイヤーは、`input_dim `引数を受け取り、この引数は`__init__()`の重み`w`および`b` の形状を計算するために使用されました。

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


多くの場合、入力値は事前に分からないので、その値が分かった時に後から（レイヤーをインスタンス化した後など）重みを定義したいと思うこともあるでしょう。

Keras APIでは、以下のようにレイヤーの`build(self, inputs_shape)`メソッドでレイヤーの重みを定義することをお勧めします。

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


レイヤーの`__call__()`メソッドは、最初に呼び出されたときに自動的にビルドを実行します。後から重みを定義できる使いやすいレイヤーができました。

In [ ]:
# At instantiation, we don't know on what inputs this is going to get called
linear_layer = Linear(32)

# The layer's weights are created dynamically the first time the layer is called
y = linear_layer(x)

## レイヤーは再帰的に構成可能

レイヤーインスタンスを別のレイヤーの属性として割り当てると、外部レイヤーは内部レイヤーの重みを追跡し始めます。

そのようなサブレイヤーは、`__init__()`メソッドで作成することをお勧めします（通常、サブレイヤーにはビルドメソッドがあるため、外部レイヤーがビルドされるときにサブレイヤーがビルドされます）。

In [ ]:
# Let's assume we are reusing the Linear class
# with a `build` method that we defined above.


class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

## `add_loss()`メソッド

レイヤーの`call()`メソッドを記述する際、トレーニングループを作成するときに、後で使用する損失テンソルを作成できます。これは、`self.add_loss(value)`を呼び出すことで実行できます。

In [ ]:
# A layer that creates an activity regularization loss
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs


これらの損失（内部レイヤーによって作成されたものを含む）は、 `layer.losses`で取得できます。このプロパティは、すべての`__call__()`の開始時にトップレベルレイヤーにリセットされるので、 `layer.losses` には、最後のフォワードパス時に定義された損失値が常に含まれます。

In [ ]:
class OuterLayer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)

    def call(self, inputs):
        return self.activity_reg(inputs)


layer = OuterLayer()
assert len(layer.losses) == 0  # No losses yet since the layer has never been called

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1  # We created one loss value

# `layer.losses` gets reset at the start of each __call__
_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1  # This is the loss created during the call above

さらに、`loss`プロパティには、内部レイヤーの重みに対して定義された正則化損失も含まれます。

In [ ]:
class OuterLayerWithKernelRegularizer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayerWithKernelRegularizer, self).__init__()
        self.dense = keras.layers.Dense(
            32, kernel_regularizer=tf.keras.regularizers.l2(1e-3)
        )

    def call(self, inputs):
        return self.dense(inputs)


layer = OuterLayerWithKernelRegularizer()
_ = layer(tf.zeros((1, 1)))

# This is `1e-3 * sum(layer.dense.kernel ** 2)`,
# created by the `kernel_regularizer` above.
print(layer.losses)

これらの損失は、以下のようにトレーニングループを記述するときに考慮されることを意図しています。

```python
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Iterate over the batches of a dataset.
for x_batch_train, y_batch_train in train_dataset:
  with tf.GradientTape() as tape:
    logits = layer(x_batch_train)  # Logits for this minibatch
    # Loss value for this minibatch
    loss_value = loss_fn(y_batch_train, logits)
    # Add extra losses created during this forward pass:
    loss_value += sum(model.losses)

  grads = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))
```

トレーニングループの記述に関する詳細は、[「トレーニングループを新規作成するためのガイド」](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch/)を参照してください。

これらの損失は、`fit()` とシームレスに連携します（自動的に集計され、メジャーな損失がある場合にはそれに追加されます）。

In [ ]:
import numpy as np

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

# If there is a loss passed in `compile`, thee regularization
# losses get added to it
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# It's also possible not to pass any loss in `compile`,
# since the model already has a loss to minimize, via the `add_loss`
# call during the forward pass!
model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

## `add_metric()`メソッド

`add_loss()`と同様に、レイヤーでは`add_metric()`メソッドでトレーニング中の数量の移動平均を追跡できます。

「ロジスティックエンドポイント」レイヤーでは、入力として予測とターゲットを受け取り、 `add_loss()`を介して追跡する損失を計算し、`add_metric()`を介して追跡する精度スカラーを計算します。

In [ ]:
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)

        # Log accuracy as a metric and add it
        # to the layer using `self.add_metric()`.
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # Return the inference-time prediction tensor (for `.predict()`).
        return tf.nn.softmax(logits)


この方法で追跡されるメトリクスは、` layer.metrics `で取得できます。

In [ ]:
layer = LogisticEndpoint()

targets = tf.ones((2, 2))
logits = tf.ones((2, 2))
y = layer(targets, logits)

print("layer.metrics:", layer.metrics)
print("current accuracy value:", float(layer.metrics[0].result()))

`add_loss()`の場合と同様に、これらのメトリックは`fit()`により追跡されます。

In [ ]:
inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

## レイヤーのシリアル化をオプションで有効化

カスタムレイヤーを[Functionalモデル](https://www.tensorflow.org/guide/keras/functional/)の一部としてシリアル化可能にする必要がある場合は、オプションで`get_config()` メソッドを実装できます。

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        return {"units": self.units}


# Now you can recreate the layer from its config:
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

基底 `Layer`クラスの`__init__()`メソッドでは、`name`や`dtype`などのキーワード引数を使えます。これらの引数を `__init__()` で親クラスに渡し、レイヤーコンフィギュレーションに含めることをお勧めします。

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config


layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

構成からレイヤーを逆シリアル化するときに柔軟性が必要な場合は、 `from_config()` クラスメソッドをオーバーライドすることもできます。これは、`from_config()`の基本実装です。

```python
def from_config(cls, config):   return cls(**config)
```

シリアル化と保存の詳細については、[「モデルの保存とシリアル化のガイド」](https://www.tensorflow.org/guide/keras/save_and_serialize/)を参照してください。

## `call()` メソッドの特権付き `トレーニング`引数

一部のレイヤー、特に`BatchNormalization`レイヤーと`Dropout`レイヤーでは、トレーニングと推論での動作が異なります。このようなレイヤーでは、`call()` メソッドで`トレーニング`引数（ブール値）を公開するのが標準的なメソッドです。

この引数を `call()`で公開することで、組み込みのトレーニングループと評価ループ（`fit()`など）を有効にして、トレーニングと推論でレイヤーを正しく使用できます。

In [ ]:
class CustomDropout(keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs


## `call()` メソッドの特権付き `マスク`引数

`call()`がサポートするもう１つの特権引数は、`マスク`引数です。

これはすべてのKeras RNNレイヤーにあります。 マスクは、時系列データを処理するときに特定の入力タイムステップをスキップするために使用されるブールテンソルです。（入力するタイムステップごとに1つのブール値を返します）。

前のレイヤーによってマスクが生成されると、Kerasは、それをサポートするレイヤーの正しい`マスク` 引数を自動的に `__call__()`に渡します。 マスク生成レイヤーは、`mask_zero=True`で構成された`Embedding`レイヤーと、`Masking` レイヤーです。

マスキングの詳細と、マスキングが有効なレイヤーの記述については、 [「マスキングとパディングのガイド」](https://www.tensorflow.org/guide/keras/masking_and_padding/)を参照してください。

## `モデル`クラス

一般的に、`レイヤー`クラスを使用して内部計算ブロックを定義し、`モデル`クラスを使用して外部モデル（トレーニングするオブジェクト）を定義します。

たとえば、ResNet50モデルでは、`レイヤー`をサブクラス化する複数のResNetブロックと、ResNet50ネットワーク全体を包含する１つの `モデル`があります。

`モデル`クラスは`レイヤー`クラスと同じAPIをもちますが、以下の点で異なります。

- 組み込みのトレーニング、評価、予測ループを公開します(`model.fit()`、`model.evaluate()`、`model.predict()`)。
- `model.layers`プロパティを介して、その内部レイヤーのリストを公開します。
- 保存およびシリアル化APIを公開します(`save()`、`save_weights()`...)

実質的に`レイヤー`クラスは、文献で「レイヤー」（「畳み込みレイヤー」または「リカレントレイヤー」など）または「ブロック」（「ResNetブロック」または「Inceptionブロック」など）と呼ばれているものに対応します。

一方、`モデル`クラスは、文献で「モデル」（「ディープラーニングモデル」など）または「ネットワーク」（「ディープニューラルネットワーク」など）と呼ばれているものに対応します 。

`レイヤー`クラスまたは`モデル`クラスのどちらを使用すべきか迷っている場合は、次の点を確認してください。 `fit()`や`save()`を呼び出す必要がある場合は、`モデル`を使用してください。 その他の場合（クラスがより大きなシステムの単なるブロックである場合や自分でトレーニングを記述してコードを保存する場合）は、`レイヤー`を使用します。

たとえば、上のmini-resnetの例を使用して`モデル` を構築し、`fit()`でトレーニングし、`save_weights()`で保存できます。

```python
class ResNet(tf.keras.Model):

    def __init__(self):
        super(ResNet, self).__init__()
        self.block_1 = ResNetBlock()
        self.block_2 = ResNetBlock()
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = Dense(num_classes)

    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.global_pool(x)
        return self.classifier(x)


resnet = ResNet()
dataset = ...
resnet.fit(dataset, epochs=10)
resnet.save(filepath)
```

## まとめ：エンドツーエンドの例

今まで学んだことを復習しましょう。

- `レイヤー`は、状態（`__init__()`または `build()`で作成）といくつかの計算（`call()`で定義）をカプセル化します。
- レイヤーを再帰的にネストし、新しい大きな計算ブロックを作成できます。
- レイヤーは、`add_loss()`および `add_metric()`を介して、損失（通常は正則化損失）およびメトリックを作成および追跡できます。
- トレーニングの対象となる外部コンテナは、`モデル`です。`モデル`は`レイヤー`に似ていますが、トレーニングとシリアル化ユーティリティが追加されます。

これらすべてをエンドツーエンドの例にまとめましょう。Variational AutoEncoder (VAE)を実装し、 MNISTディジットでトレーニングします。

VAEは`モデル`のサブクラスで、` Layer `をサブクラス化するLayerのネストされた構成として構築されます。 正則化損失（KLダイバージェンス）が特徴です。

In [ ]:
from tensorflow.keras import layers


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed


MNISTで簡単なトレーニングループを書いてみましょう。

In [ ]:
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

VAEは`Model`をサブクラス化しているため、組み込みのトレーニングループが備わっています。 そのため、次のようにトレーニングすることもできます。

In [ ]:
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=2, batch_size=64)

## オブジェクト指向開発を超えて：Functional API

ここでは、オブジェクト指向の開発の例を多く紹介しましたが、 [Functional API](https://www.tensorflow.org/guide/keras/functional/)を使用してモデルを構築することもできます。重要なのは、いずれかのスタイルを選択しても、もう一方のスタイルで記述されたコンポーネントを組み合わせることができるということです。

たとえば、以下のFunctional APIの例では、上の例で定義した `Sampling` レイヤーを再利用しています。

In [ ]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

# Train.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

詳細につきましては、[Functional API のガイド](https://www.tensorflow.org/guide/keras/functional/)を参照してください。